<a href="https://colab.research.google.com/github/Hernanros/SOTA/blob/master/notebooks/labels/L2_elmo_pickle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install wmd

     |████████████████████████████████| 112kB 2.7MB/s 
  Created wheel for wmd: filename=wmd-1.3.2-cp36-cp36m-linux_x86_64.whl size=629397 sha256=e7f52dec50a1e6425db2700d50bc5a30adc1d4847d8dc0303e34ce83d18503d7
  Stored in directory: /root/.cache/pip/wheels/a6/4d/7a/fc3fdbc12e33c0551301543e4441fb4783c809a30451387ffd
Successfully built wmd


In [2]:
!pip install wordfreq
!git clone https://github.com/brmson/dataset-sts
# !wget  http://nlp.stanford.edu/data/glove.840B.300d.zip

     |████████████████████████████████| 32.8MB 93kB/s 
     |████████████████████████████████| 4.9MB 47.4MB/s 
     |████████████████████████████████| 276kB 52.9MB/s 
  Created wheel for wordfreq: filename=wordfreq-2.3.2-cp36-none-any.whl size=32817238 sha256=52429bfa7ee0ab92aa4a4c62bbf1935f570a28f34a0a776ca42c281f4557cf54
  Stored in directory: /root/.cache/pip/wheels/8d/ba/84/ba6be76208bd2c2124b6586f7967fb87e9f9fb4b4827e5e2c9
  Created wheel for langcodes: filename=langcodes-2.0.0-cp36-none-any.whl size=5044047 sha256=5f7a9de602b0d7592e23be7d750fe5b56d6c83d87642360ce74aa8bcada19aa6
  Stored in directory: /root/.cache/pip/wheels/c9/11/90/c7bba8118f3674d75e1457537635266a12538cf622a4684bb2
  Created wheel for marisa-trie: filename=marisa_trie-0.7.5-cp36-cp36m-linux_x86_64.whl size=862227 sha256=b0bb43c8cf297c7b09db817ebdda0aca84519db68d507bbd03b4252e948d967e
  Stored in directory: /root/.cache/pip/wheels/45/24/79/022624fc914f0e559fe8a1141aaff1f9df810905a13fc75d57
Successfully built word

In [2]:
% cd /content/drive/My Drive/NLP_Project

/content/drive/My Drive/NLP_Project


In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
# from wmd import WMD
import wordfreq
import pickle

In [7]:
zip_ref = zipfile.ZipFile("./glove.840B.300d.zip")
zip_ref.extractall("/tmp")
zip_ref.close()

In [9]:
glove_pd = pd.read_csv('glove_pd.csv',index_col=0)
#glove_pd = pd.read_csv(open('glove.840B.300d.txt'), sep=" ", quoting=3, header=None, index_col=0)
glove = {key: val.values for key, val in glove_pd.T.items()}
#glove_pd.to_csv('/content/drive/My Drive/NLP_Project/glove_pd.csv')
# np.savetxt(r'/content/drive/My Drive/NLP_Project/glove.txt', glove_pd.values, fmt='%d', delimiter=' ')

In [4]:
with open('combined_data.pickle', 'rb') as handle:
    df = pickle.load(handle)


In [5]:
tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize(df.iloc[9,0])
df['text_1_tokens'] =df.text_1.apply(lambda x:tokenizer.tokenize(x)) 
df['text_2_tokens'] =df.text_2.apply(lambda x:tokenizer.tokenize(x)) 

def embed (sent):
  toklist = tokenizer.tokenize(sent)
  embs = []
  freqs = []
  for word in toklist:
    try: 
      embs.append(glove[word.lower()])
      freqs.append(wordfreq.word_frequency(word, 'en', wordlist='large'))
    except:
      try:
        embs.append(glove[word.strip("'")])
        freqs.append(wordfreq.word_frequency(word, 'en', wordlist='large'))

      except:
        continue
  return embs, freqs

embed(df.iloc[9,0])
text_1_embeddings = df.text_1.apply(lambda x: embed(x) )
text_2_embeddings = df.text_2.apply(lambda x: embed(x) )

# Elmo l2

In [6]:
! pip install flair==0.5
! pip install allennlp==0.9

In [11]:

from flair.embeddings import ELMoEmbeddings
from flair.data import Sentence
from scipy.spatial.distance import euclidean
import torch

In [8]:

# init embedding
embedding = ELMoEmbeddings()


In [34]:
sent = Sentence(df['text_1'].iloc[0])
emb = embedding.embed(sent)
torch.stack([token.embedding for token in sent]).mean(axis = 1)

torch.Size([15])

In [35]:
def create_embedding(sentence):
    # embed words in sentence
    sent = Sentence(sentence)
    embedding.embed(sent)
    # return average embedding of words in sentence
    return torch.stack([token.embedding for token in sent]).mean(axis=0)

In [46]:
euclidean(np.array(create_embedding(df.text_1.iloc[255]).cpu()),np.array(create_embedding(df.text_2.iloc[255]).cpu()))

12.855411529541016

In [55]:
from tqdm import tqdm
s1,s2 =[], []
pbar = tqdm(total = len(df), leave = True, position = 0)
for i in range( len(df)):
  s1.append(create_embedding(df.text_1.iloc[i]))
  s2.append(create_embedding(df.text_2.iloc[i]))
  pbar.update()
pbar.close()



100%|██████████| 25767/25767 [33:25<00:00, 12.85it/s]


In [60]:
s1,s2 = [np.array(s1[i].cpu()) for i in range(len(s1))],[np.array(s2[i].cpu()) for i in range(len(s2))]


array([tensor([ 0.1771,  0.1509, -0.1012,  ...,  0.1897, -0.7050,  0.1215],
       device='cuda:0'),
       tensor([-0.1970, -0.1155, -0.0693,  ..., -0.3429, -0.7645, -0.0062],
       device='cuda:0'),
       tensor([ 0.0238,  0.0237, -0.0225,  ...,  0.1077,  0.6887,  0.3335],
       device='cuda:0'),
       ...,
       tensor([-3.4835e-02,  2.4674e-04,  7.2018e-02,  ...,  4.4446e-01,
         8.7405e-01, -2.1138e-01], device='cuda:0'),
       tensor([-0.1024, -0.2074, -0.1226,  ...,  0.3130,  0.0981,  0.2823],
       device='cuda:0'),
       tensor([ 0.1499, -0.2152,  0.0116,  ...,  0.2606,  1.2691,  0.2507],
       device='cuda:0')], dtype=object)

In [70]:
df['L2_score'] = [euclidean(s1[i],s2[i]) for i in range(len(s1))]
df.head()

,dataset,label,text_1,text_2,bleu_allwords,bleu_withoutstop,glove_allwords,glove_withoutstop,ftext_allwords,ftext_withoutstop,WMD,1-gram_overlap,2-gram_overlap,3-gram_overlap,4-gram_overlap,ROUGE-1 recall,ROUGE-1 precision,ROUGE-1 F,ROUGE-2 recall,ROUGE-2 precision,ROUGE-2 F,ROUGE-L recall,ROUGE-L precision,ROUGE-L F,chrf_score,chrf_score_norm,POS dist score,text_1_tokens,text_2_tokens,L2_score
0,2012.MSRpar.test.tsv,4.4,The problem likely will mean corrective change...,He said the problem needs to be corrected befo...,0.375739,0.333333,96.20,90.82,77.23,77.39,3,0.400000,0.133333,0.000000,0.000000,0.466667,0.368421,0.411765,0.214286,0.166667,0.187500,0.466667,0.368421,0.411765,0.536815,2.684077,3.055075,"[The, problem, likely, will, mean, corrective,...","[He, said, the, problem, needs, to, be, correc...",10.527886
1,2012.MSRpar.test.tsv,0.8,The technology-laced Nasdaq Composite Index .I...,The broad Standard & Poor 's 500 Index .SPX in...,0.461538,0.333333,95.66,88.96,87.95,72.39,3,0.400000,0.117647,0.000000,0.000000,0.500000,0.428571,0.461538,0.176471,0.150000,0.162162,0.500000,0.428571,0.461538,0.391649,1.958247,3.042741,"[The, technology, laced, Nasdaq, Composite, In...","[The, broad, Standard, Poor, s, 500, Index, SP...",11.930013
2,2012.MSRpar.test.tsv,3.6,"`` It 's a huge black eye , '' said publisher ...","`` It 's a huge black eye , '' Arthur Sulzberg...",0.578947,0.500000,97.75,94.94,88.39,72.44,2,0.647059,0.294118,0.250000,0.200000,0.640000,0.727273,0.680851,0.333333,0.380952,0.355556,0.560000,0.636364,0.595745,0.549542,2.747710,4.329347,"[It, s, a, huge, black, eye, said, publisher, ...","[It, s, a, huge, black, eye, Arthur, Sulzberge...",9.865285
3,2012.MSRpar.test.tsv,3.4,SEC Chairman William Donaldson said there is a...,`` I think there 's a building confidence that...,0.578947,0.444444,98.17,90.51,94.16,75.97,2,0.689655,0.516129,0.413793,0.296296,0.636364,0.823529,0.717949,0.428571,0.562500,0.486486,0.590909,0.764706,0.666667,0.547146,2.735730,0.000000,"[SEC, Chairman, William, Donaldson, said, ther...","[I, think, there, s, a, building, confidence, ...",10.122802
4,2012.MSRpar.test.tsv,1.4,Vivendi shares closed 1.9 percent at 15.80 eur...,"In New York , Vivendi shares were 1.4 percent ...",0.384615,0.375000,95.77,87.00,90.74,67.48,3,0.322581,0.066667,0.000000,0.000000,0.294118,0.357143,0.322581,0.062500,0.076923,0.068966,0.294118,0.357143,0.322581,0.293020,1.465099,0.000000,"[Vivendi, shares, closed, 1, 9, percent, at, 1...","[In, New, York, Vivendi, shares, were, 1, 4, p...",14.036093


In [71]:
with open('combined_data.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)